In [10]:
!pip install tensorflow_datasets

/bin/sh: module: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `module'
/bin/sh: scl: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `scl'
/bin/sh: scl: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `scl'
/bin/sh: module: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `module'
     |████████████████████████████████| 3.4 MB 8.0 MB/s 
     |████████████████████████████████| 68 kB 3.1 MB/s 
     |████████████████████████████████| 177 kB 17.2 MB/s 
  Using cached future-0.18.2.tar.gz (829 kB)
     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 100 kB 2.4 MB/s 
  Created wheel for dill: filename=dill-0.3.2-py3-none-any.whl size=78912 sha256=361ed2308fdabcf99e980d5bd0046b17d5dd7bd26c224c326eb0538d8784d08c
  Stored in directory: /rds/general/user/kk2219/home/.cac

In [12]:
import os
import os.path
from os import path

import numpy as np
np.random.seed(0)
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from pylab import *
from sklearn.metrics import roc_curve,roc_auc_score
from datetime import datetime,timedelta
#import wandb
#from GenericTools import *
#from TripletLossTools import *
#from QuadrupletLossTools import *


from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform

from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
import tensorflow as tf
import tensorflow_datasets as tfds

# Allow memory growth for the GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
def dataSetToArrays(nb_classes,datasets, splitname):
    '''
    Organise the tensorflow dataset into numpy arrays
    Inputs
        nb_classes : global number of classes
        datasets : tensorflow datasets "pack" as they are in the tensorflow_datasets repo
        splitname : which split name from the tfds to take
    Output
        dataset : list of real_nb_class element, each containing numpy array images for each class
        dataset_flat_X : numpy array images of all classes
        dataset_flat_Y : numpy array true labels of all classes
        real_nb_class : number of class detected in this split
    '''
    dataset = []
    tempdataset = []
    dataset_flat_X = []
    dataset_flat_Y = []
    real_nb_class = 0
    
    for n in range(nb_classes):
        tempdataset.append([])
    
    for row in tfds.as_numpy(datasets[splitname]):
       
        
        #Add in the flat dataset
        dataset_flat_X.append(row['image'])
        dataset_flat_Y.append(row['label'])
                
        #Add in the sorted dataset
        tempdataset[row['label']].append(row['image'])
        
    
    #Transform into nparrays, inverted grayscale and normalize
    for n in range(nb_classes):
        if len(tempdataset[n])>1:
            dataset.append(np.mean(np.asarray(tempdataset[n]), axis=-1, keepdims=True)/-255+1)
            real_nb_class += 1
    
    #idem
    dataset_flat_X = np.mean(np.asarray(dataset_flat_X), axis=-1, keepdims=True)/-255+1
    dataset_flat_Y = np.asarray(dataset_flat_Y)
    
    return dataset,dataset_flat_X,dataset_flat_Y,real_nb_class

def buildDataSet():
    omniglot = tfds.builder('omniglot')
    input_shape = omniglot.info.features['image'].shape
    nb_classes = omniglot.info.features['label'].num_classes
    omniglot.download_and_prepare()
    datasets = omniglot.as_dataset()
    
    #Dataset sorted by class
    dataset_train,dataset_train_flat_X,dataset_train_flat_Y,nb_classes_train = dataSetToArrays(nb_classes,datasets,'train')
    dataset_test, dataset_test_flat_X, dataset_test_flat_Y,nb_classes_test  = dataSetToArrays(nb_classes,datasets,'test')
    
    return dataset_train,dataset_test, \
            dataset_train_flat_X,dataset_train_flat_Y,dataset_test_flat_X,dataset_test_flat_Y, \
            nb_classes,nb_classes_train,nb_classes_test

In [14]:
dataset_train,dataset_test, \
    dataset_train_flat_X,dataset_train_flat_Y,dataset_test_flat_X,dataset_test_flat_Y, \
    nb_classes, nb_classes_train,nb_classes_test = buildDataSet()
img_rows, img_cols = dataset_train[0].shape[1],dataset_train[0].shape[2]
input_shape = (img_rows, img_cols, 1)
print("{} classes, {} for training, {} for testing".format(nb_classes, nb_classes_train,nb_classes_test))
print("Checking shapes for class 0 (train) : ",dataset_train[0].shape)
print("Checking shapes for class 0 (test) : ",dataset_test[0].shape)
print("Checking shape for flat train dataset X and Y", dataset_train_flat_X.shape,dataset_train_flat_Y.shape)
print("Checking shape for flat test  dataset X and Y", dataset_test_flat_X.shape,dataset_test_flat_Y.shape)
print("Checking first samples")
for i in range(2):
    DrawPics(dataset_train[i],5,template='Train {}',classnumber=i)
    DrawPics(dataset_test[i],5,template='Test {}',classnumber=i)

TypeError: __init__() missing 2 required positional arguments: 'op' and 'message'